In [302]:
# -*- coding: utf-8 -*-
import csv
from StringIO import StringIO
from collections import namedtuple
import random
import sys
reload(sys)
sys.setdefaultencoding('UTF8')


filePath="daily_clickStreamData.csv"
dailyData = sc.textFile(filePath)


In [303]:
def parse(row):
    row = [s.encode('utf-8') for s in StringIO(row)]
    reader = csv.reader(row)
    row = reader.next()
    return DailyClickData(*row)
def conv(x):
    x = [s.encode('utf-8') for s in StringIO(x)]
    for a in x:
        a = a.replace('İ', 'I')
        a = a.replace('Ş', 's')
        a = a.replace('Ç', 'C')
        a = a.replace('Ü', 'U')
        a = a.replace('Ö', 'O')
        a = a.replace('ş', 's')
        a = a.replace('ğ', 'g')
        a = a.replace('ı', 'i')
        a = a.replace('ö', 'o')
        a = a.replace('ü', 'u')
        a = a.replace('ç', 'c')
    return str(a).lower()

In [304]:
header = dailyData.first()
dataWoHeader = dailyData.filter(lambda x : x<>header)
fields = header.replace(" ","_").replace("/","_").split(",")
# we give name every column

DailyClickData = namedtuple('DailyClickData',fields, verbose=True ) 
daily_click_data = dataWoHeader.map(parse)

In [305]:
#for limitted time.
citys_of_the_HURRIYET_dailyClickStream = daily_click_data.map( lambda x : (x.city,'1') )
citys_of_the_HURRIYET_dailyClickStream.take(4)

[('ankara', '1'), ('bonn', '1'), ('bolu', '1'), ('van', '1')]

In [306]:
#in this file we have cities location with names
filePath_city_locations_in_turkey = "citys.csv"
city_locations_in_turkey = sc.textFile(filePath_city_locations_in_turkey)
city_locations_in_turkey = city_locations_in_turkey.map(lambda x : x.split(";"))
city_locations_in_turkey = city_locations_in_turkey.map(lambda x : ( conv(x[0]), ( x[1].replace(",",".") +"," +x[2].replace(",",".") ) ) ) 
city_locations_in_turkey.take(4)

[('adana', u'37,35.321333'),
 ('adiyaman', u'37.764751,38.278561'),
 ('afyonkarahisar', u'38.750714,30.556692'),
 ('agri', u'39.626922,43021596')]

In [311]:
#The coordinates we have are real(coordinates of the citys). so we changed the data to distribute a little bit.
def giveRndCLa(value):
    number = float (value.split(",")[0])
    rndNumber = number + (0.03*random.randint(0,9)) + (0.0001*random.randint(0,9)) + (0.001*random.randint(0,9) )
    return rndNumber
def giveRndCLo(value):
    number = float (value.split(",")[1])
    rndNumber = number + (0.03*random.randint(0,9)) + (0.0001*random.randint(0,9)) + (0.001*random.randint(0,9) )
    return rndNumber


In [312]:
city_trend = city_locations_in_turkey.leftOuterJoin(citys_of_the_HURRIYET_dailyClickStream)
city_trend.take(5)

[('zonguldak', (u'41.456409,31.798731', '1')),
 ('zonguldak', (u'41.456409,31.798731', '1')),
 ('zonguldak', (u'41.456409,31.798731', '1')),
 ('zonguldak', (u'41.456409,31.798731', '1')),
 ('zonguldak', (u'41.456409,31.798731', '1'))]

In [313]:
lon = list (city_trend.map(lambda x :  giveRndCLa(x[1][0]) ).collect())
lag = list (city_trend.map(lambda x :  giveRndCLo(x[1][0]) ).collect())

In [314]:
import gmplot

#zoom coordinate
gmap = gmplot.GoogleMapPlotter(39.92077,32.85411, 6)

# Overlay our datapoints onto the map
for i in range(0,5):
    gmap.heatmap(lon, lag)
# Generate the heatmap into an HTML file
gmap.draw("my_heatmap.html")